In [2]:
import ccxt
import numpy as np
import pandas as pd
import time, datetime
import threading
import configparser
from glob import glob
import os
from math import ceil

from data_provider import get_symbols, read_data_path, market_to_symbol, create_klines

pd.set_option('precision', 9)

In [3]:
exchange = ccxt.binance()
pair = 'BTC'
now = exchange.milliseconds()
to_timestamp = now

timeframes = {'1m':1, '3m':3, '5m':5, '15m':15, '30m':30, '1h':60, '2h':120, '4h':240, '6h':480, '12h':960, '1d':1920}
#timeframes = {'1h':60}

#exchange.timeframes.values()

symbol_list = get_symbols(exchange, pair)
markets = [x.replace('/','-') for x in symbol_list]
#fast_trend = ['ADA-BTC', 'EOS-BTC', 'TRX-BTC', 'ONT-BTC', 'NEO-BTC','ZIL-BTC', 'XRP-BTC'] #april 2018
#markets = fast_trend
#markets = ['BTC-USDT']
#markets = ['QKC-BTC']

data_path = read_data_path() + '/' + pair

In [7]:
#Обновление данных
if not os.path.isdir(data_path):
    print("Директория отсутствует")

start_timestamp = exchange.parse8601('2018-01-01 00:00:00')
print(start_timestamp)
start_time = time.time()

for market in markets:
    print(market)
    symbol = market_to_symbol(market)
    folders_exist = next(os.walk(data_path))[1]

    if market not in folders_exist:
        newpath = data_path + '/' + market
        os.makedirs(newpath)

    folders_exist = next(os.walk(data_path + '/' + market))[1]

    tf_keys = list(timeframes.keys())
    for tf in tf_keys:
        print(tf)
        tf_path = data_path + '/' + market +'/' + tf 
        if tf not in folders_exist:
            os.makedirs(tf_path)
        
        files = glob(tf_path + '/*.csv')
        from_timestamp = start_timestamp
        if files != []:
            df_old   = pd.read_csv(files[-1])
            from_timestamp = int(df_old.iloc[-1]['T'])
        else:
            df_old   = pd.DataFrame(columns = ['T', 'O', 'H', 'L', 'C', 'V'])
            
        data = []        
        while from_timestamp < to_timestamp:
            time.sleep (exchange.rateLimit / 1000) # time.sleep wants seconds
            
            try:
                ohlcvs = exchange.fetch_ohlcv (symbol, tf, from_timestamp)  
            except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout) as error:
                #error_log = exchange.iso8601(now), 'Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...'
                error_log = str(exchange.iso8601(now)), 'Got an error', type(error).__name__, ', retrying in', str(hold), 'seconds...'
                error_log = ' '.join(error_log)
                print(error_log)
                time.sleep(hold)  

         
            if ohlcvs == []:
                continue
            first  = ohlcvs[0][0]
            last   = ohlcvs[-1][0]
            #print('\t First candle epoch', first, exchange.iso8601(first))
            #print('\t Last candle epoch', last, exchange.iso8601(last), '\n')

            from_timestamp = first  # На случай если на заданную дату данных еще нет
            from_timestamp += len(ohlcvs) * timeframes[tf] * 1000 * 60
            data += ohlcvs

        if data != []:
            df_new = pd.DataFrame(data)
            df_new.columns = ['T', 'O', 'H', 'L', 'C', 'V']
            df = pd.concat([df_old, df_new])
            df = df.drop_duplicates()
            file_name = tf_path + '/' + market + '_' + str(last) + '.csv'
            df.to_csv(file_name, index = False)  
        
        files = glob(tf_path + '/*.csv')
        if len(files) > 1 : 
            os.remove(files[0])
    
print('Done')
time_taken = (time.time() - start_time)/60/60
print("Time taken = {0:.3f}".format(time_taken),' hours')

1514764800000
ADA-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
ADX-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
AE-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
AGI-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
AION-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
AMB-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
APPC-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
ARDR-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
ARK-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
ARN-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
AST-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BAT-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BCD-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BCH-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BCN-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BCPT-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BLZ-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BNB-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BNT-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BQX-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BRD-BTC
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BTC-USDT
15m
12h
5m
6h
3m
4h
1d
2h
1m
30m
1h
BTG-BTC
15m
12

ExchangeError: binance https://api.binance.com/api/v1/klines?startTime=1533700800000&interval=15m&symbol=HSRBTC GET ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [6]:
ohlcvs

[]

In [80]:
tf = 24
create_klines(data_path, markets, tf)

ADA-BTC
EOS-BTC
TRX-BTC
ONT-BTC
NEO-BTC
ZIL-BTC
XRP-BTC


In [ ]:
symbol = 'ADA/USDT'
tf = '1m'

exchange.fetch_ohlcv (symbol, tf, start_timestamp)

In [ ]:
if not os.path.isdir(data_path):
    print("Директория отсутствует")


start_time = time.time()

for market in markets:
    print(market)
    symbol = market_to_symbol(market)
    folders_exist = next(os.walk(data_path))[1]

    if market not in folders_exist:
        newpath = data_path + '/' + market
        os.makedirs(newpath)

    folders_exist = next(os.walk(data_path + '/' + market))[1]

    tf_keys = list(timeframes.keys())
    for tf in tf_keys[:6]:
        print(tf)
        if tf not in folders_exist:
            newpath = data_path + '/' + market +'/' + tf 
            os.makedirs(newpath)

        data = []
        from_timestamp = exchange.parse8601(from_datetime)
        while from_timestamp < to_timestamp:
            time.sleep (exchange.rateLimit / 1000) # time.sleep wants seconds
            ohlcvs = exchange.fetch_ohlcv (symbol, tf, from_timestamp)    
            first = ohlcvs[0][0]
            last = ohlcvs[-1][0]
            #print('\t First candle epoch', first, exchange.iso8601(first))
            #print('\t Last candle epoch', last, exchange.iso8601(last), '\n')

            from_timestamp = first  # На случай если на заданную дату данных еще нет
            from_timestamp += len(ohlcvs) * timeframes[tf] * 1000 * 60
            data += ohlcvs


        df = pd.DataFrame(data)
        df.columns = ['T', 'O', 'H', 'L', 'C', 'V']
        file_name = data_path + '/' + market + '/' +  tf + '/' + market + '_' + str(last) + '.csv'
        df.to_csv(file_name, index = False)    
    
print('Done')
time_taken = (time.time() - start_time)/60/60
print("Time taken = {0:.3f}".format(time_taken),' hours')

In [26]:
fn = data_path + '/' + market + '/' + tf
print(fn)
files = glob(fn + '/*.csv')
df = pd.read_csv(files[-1])
t1 = df.iloc[1]['T']
print(exchange.iso8601())

/home/user/Robots/Data/Binance/USDT/XRP-USDT/5m
2018-05-04T08:15:00.000Z


In [ ]:
   
def data_read(exchange, tick, restart = 0):
    data_path = read_data_path()

    with open("currencies.txt", 'r') as file:
        content = file.read()
        saving_currencies = content.split("\n") #split it into lines
    file.closed

    #Проверяем наличие директорий для сохранения файлов, создаем в случае отсутсвия
    if not os.path.isdir(data_path):
        print("Директория отсутствует")
        return 0
        
    folders_exist = next(os.walk(data_path))[1]
    folders_to_make = list(set(saving_currencies) - set(folders_exist))
    
    for folder in folders_to_make :
        newpath = data_path + folder +'/' + tick 
        os.makedirs(newpath)
        
    
    #Читаем и записываем данные
    dp = Bittrex(None, None, api_version= API_V2_0)

    for market_name in saving_currencies:
        print(market_name)
        folders_exist = next(os.walk(data_path + '/' + market_name))[1]
        if not (tick in folders_exist):
            newpath = data_path + market_name +'/' + tick 
            os.makedirs(newpath)
            
        dj = dp.get_candles(market_name, tick)
        df = json_normalize(dj['result'])
        last_time = df[-1:]['T'].values[0]
        last_time = last_time.replace(':','-')
        file_name = data_path + market_name + '/' +  tick + '/' + market_name + "-" + last_time + ".csv"
        df.to_csv(file_name, index = False)

720

In [40]:
import ciso8601

dt = ciso8601.parse_datetime('2018-04-01')
# to get time in seconds:
from_ts = time.mktime(dt.timetuple()) * 1000

dt = ciso8601.parse_datetime('2018-05-01')
# to get time in seconds:
to_ts = time.mktime(dt.timetuple()) * 1000

df[(df['T'] > from_ts) & (df['T'] < to_ts)]

,T,O,H,L,C,V
6217,1522530000000,0.0015740,0.0015929,0.0015738,0.0015867,49460.25
6218,1522533600000,0.0015867,0.0016139,0.0015860,0.0016066,84122.42
6219,1522537200000,0.0016045,0.0016069,0.0015897,0.0015965,40651.94
6220,1522540800000,0.0015958,0.0016100,0.0015866,0.0015884,98226.92
6221,1522544400000,0.0015884,0.0016033,0.0015868,0.0015948,69941.68
6222,1522548000000,0.0015932,0.0015990,0.0015899,0.0015937,63642.15
6223,1522551600000,0.0015922,0.0015980,0.0015825,0.0015980,75213.42
6224,1522555200000,0.0015978,0.0016100,0.0015946,0.0016054,97181.47
6225,1522558800000,0.0016067,0.0016337,0.0015953,0.0016334,114580.59
6226,1522562400000,0.0016334,0.0016337,0.0016080,0.0016120,152606.58
